# Задание 1 (10 баллов)

Обучите языковую модель с помощью GRPO на вот этом датасете - `notbadai/python_functions_reasoning`
Он чем-то похож на датасет из семинара, но вместо математических задач нужно написать код на питоне.
Измените system prompt под эту задачу. Напишите аналогичные reward функции - одна на проверку формата (markdown wrappers ```python... ``` вокруг кода в конце), и на проверку что код работает (можете использовать import ast; ast.parse(string) для проверки)
Если модель сначала не может отвечать в нужно формате, то сделайте цикл SFT обучения на небольшом куске датасета.
GRPO в колабе может работать долго, поэтому не старайтесь пропустить весь датасет. Но обучайте хотя бы на 100 промптах (по несколько генераций на каждый промпт).
Оцените разницу в качестве на небольшом сабсете, прогнав изначальную модель, модель после SFT (если она есть) и GRPO модель. В качестве метрики используйте reward функции.
Можете взять любую языковую модель любого размера. Можете дообучать всю модель или же только адаптера поверх квантизированной модели.


In [23]:
import ast

In [ ]:
!pip install -U bitsandbytes

In [ ]:
!pip install datasets

In [ ]:
!pip install trl

In [24]:
from datasets import load_dataset
from trl import GRPOConfig, GRPOTrainer


from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from trl import DPOTrainer

from tqdm.notebook import tqdm
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training

from trl import SFTConfig, SFTTrainer

import re

In [25]:
model_name = "Qwen/Qwen2-0.5B-Instruct"
device = 'cuda'

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

#### Необходимые функции:

In [27]:
system_prompt = """
Your task is to write correct and efficient python code based on the user's task. Before outputting the result, give step-by-step reasoning. Always wrap the final code in markdown code blocks like this:
```python
# your final code here
```
"""

# system prompt задает формат ответа, в модель подается текст (то есть на выходе должен получиться только промпт)
# формат: систем промпт, вопрос от пользователя, ответ из этого датасета, этот формат подается в модель в виде текста, модель просто учится генерировать такие же тексты

def chatml_format_sft(example): # просто текст (для SFT)
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": example['prompt']},
                {"role": "assistant", "content": example['reasoning'] + "\n\n" + example['answer']}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return {
        "text": prompt
    }

In [28]:
def chatml_format(example): # для всех других моделей, которые не SFT
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": example['prompt']}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    return {
        "prompt": prompt
    }

Функции вознаграждения:

In [29]:
# пишем эти функции с оглядкой на датасет (ответы в нем)

# эта функция просто проверяет формат, она ищет есть ли такой формат в ответе
def format_accuracy_func(completions, **kwargs):
    rewards = []
    for response in completions:
        if re.search(r'```python\n(?!.*# your (?:final )?code here)([\s\S]*?)\n```', response): # просто ищем такую строчку
            rewards.append(1.0)

        else:
            rewards.append(0.0)

    return rewards


# эта функция проверяет правильность ответа, в нашем случае то, что код работает
def answer_accuracy_func(completions, **kwargs):
    rewards = []
    for response in completions:
        code_block = re.search(r'```python\n(?!.*# your (?:final )?code here)([\s\S]*?)\n```', response, re.DOTALL)

        if code_block is not None:
            code = code_block.group(1).strip()
            try:
                ast.parse(code)
                rewards.append(1.0)
            except SyntaxError:
                rewards.append(0.0)
        else:
            rewards.append(0.0)

    return rewards

#### Датасет:

In [30]:
dataset = load_dataset("notbadai/python_functions_reasoning", "default")['train']

In [31]:
train = dataset.select(range(100)).map(chatml_format)
test = dataset.select(range(100, 200)).map(chatml_format)
sft_dataset = dataset.select(range(500, 1000))
sft_dataset = sft_dataset.map(chatml_format_sft, remove_columns=sft_dataset.column_names)

#### Базовая модель:

In [33]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [34]:
model = model.to(device)

In [35]:
batch = test

In [36]:
input_ids = tokenizer.batch_encode_plus(batch['prompt'], return_tensors='pt', padding=True)

output = model.generate(
    input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
    max_new_tokens=400, do_sample=True, temperature=1.5, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
)

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


In [37]:
completions_base_model = tokenizer.batch_decode(output, skip_special_tokens=False)

In [38]:
completions_base_model

['<|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end

In [39]:
print(f"Сумма по функции, которая проверяет формат: {sum(format_accuracy_func(completions_base_model))}")

Сумма по функции, которая проверяет формат: 65.0


In [40]:
print(f"Сумма по функции, которая проверяет, работает ли код: {sum(answer_accuracy_func(completions_base_model))}")

Сумма по функции, которая проверяет, работает ли код: 59.0


#### SFT:

In [54]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear"
)

In [55]:
model = get_peft_model(model, peft_config)

In [49]:
sft_dataset[0]['text']

'<|im_start|>system\n\nYour task is to write correct and efficient python functions. Before outputting the result, give step-by-step reasoning. Always wrap the final code in markdown code blocks like this:\n```python\n# your final code here\n```\n<|im_end|>\n<|im_start|>user\n```python\ndef find_missing_positive_integer(sequence):\n    """Write a function to find the smallest positive integer that does not occur in a given sequence of integers. The function should efficiently handle sequences with both positive and negative integers, as well as sequences with duplicates.\n    """\n```<|im_end|>\n<|im_start|>assistant\nTo solve this problem, we need to find the smallest positive integer that is not present in a given sequence of integers. Here is a step-by-step plan:\n\n1. **Understand the Problem**:\n   - The sequence contains both positive and negative integers.\n   - There might be duplicates in the sequence.\n   - We need to find the smallest positive integer that is not in the sequ

In [56]:
training_args = SFTConfig(
    max_length=200,
    report_to="none",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    logging_steps=10
)

In [57]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=sft_dataset,
    args=training_args
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [58]:
trainer.train()

Step,Training Loss
10,2.084700
20,1.582900
30,1.291800
40,0.999200
50,0.893400
60,0.857700
70,0.813000
80,0.769100
90,0.749800
100,0.667800


TrainOutput(global_step=375, training_loss=0.7587793947855631, metrics={'train_runtime': 434.4703, 'train_samples_per_second': 3.452, 'train_steps_per_second': 0.863, 'total_flos': 675890150400000.0, 'train_loss': 0.7587793947855631})

In [59]:
trainer.save_model("Qwen/Qwen2-0.5B-sft")

In [60]:
model_name = "Qwen/Qwen2-0.5B-sft"
# device = 'cuda'

In [61]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
).to(device)

for param in model.parameters():
    param.requires_grad = False

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [62]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [63]:
input_ids = tokenizer.batch_encode_plus(batch['prompt'], return_tensors='pt', padding=True)

output = model.generate(
    input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
    max_new_tokens=400, do_sample=True, temperature=1.5, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
)

In [64]:
completions_sft_model = tokenizer.batch_decode(output, skip_special_tokens=False)

In [65]:
print(f"Сумма по функции, которая проверяет формат: {sum(format_accuracy_func(completions_sft_model))}")

Сумма по функции, которая проверяет формат: 49.0


In [66]:
print(f"Сумма по функции, которая проверяет, работает ли код: {sum(answer_accuracy_func(completions_sft_model))}")

Сумма по функции, которая проверяет, работает ли код: 49.0


#### GRPO:

In [67]:
training_args = GRPOConfig(output_dir="Qwen/Qwen2-0.5B-sft-grpo",
                           logging_steps=10,
                           report_to="none",
                           num_generations=8, # сколько примеров в каждой группе GRPO, рекомендуется его меньше 8 не ставить
                           num_train_epochs=1,
                           temperature=1.5,
                           label_names=["labels"])

In [68]:
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[answer_accuracy_func, format_accuracy_func],
    args=training_args,
    train_dataset=train,
    peft_config=peft_config
)

In [69]:
trainer.train()

Step,Training Loss
10,0.006000
20,0.014800
30,0.037300
40,0.011200
50,0.041600
60,0.013200
70,0.033600
80,0.066800
90,0.021800
100,0.038100


TrainOutput(global_step=100, training_loss=0.028458391427993775, metrics={'train_runtime': 3113.5814, 'train_samples_per_second': 0.032, 'train_steps_per_second': 0.032, 'total_flos': 0.0, 'train_loss': 0.028458391427993775})

In [70]:
trainer.save_model('Qwen/Qwen2-0.5B-sft-grpo')

In [72]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-sft-grpo")
tokenizer.pad_token = tokenizer.eos_token
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

In [73]:
model_ref = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B-sft", device_map='cuda', torch_dtype=torch.bfloat16)

device = model_ref.device

In [74]:
model = PeftModel.from_pretrained(model_ref, "Qwen/Qwen2-0.5B-sft-grpo")

In [75]:
input_ids = tokenizer.batch_encode_plus(batch['prompt'], return_tensors='pt', padding=True)

output = model.generate(
    input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
    max_new_tokens=400, do_sample=True, temperature=1.5, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
)

In [76]:
completions_grpo_model = tokenizer.batch_decode(output, skip_special_tokens=False)

In [77]:
print(f"Сумма по функции, которая проверяет формат: {sum(format_accuracy_func(completions_grpo_model))}")

Сумма по функции, которая проверяет формат: 77.0


In [78]:
print(f"Сумма по функции, которая проверяет, работает ли код: {sum(answer_accuracy_func(completions_grpo_model))}")

Сумма по функции, которая проверяет, работает ли код: 74.0


#### Вывод

Базовая модель показывает вполне неплохие результаты (и в completions видно, что модель даже пытается следовать промпту). После дообучения (sft) качество хуже, что странно (оно ниже базовой модели). Но GRPO модель в любом случае показала самые лучшие результаты